In [1]:
import numpy as np 
import pickle
from matplotlib import pyplot as plt
from galpy.util import bovy_coords as coords
import os
import sys
#import superfreq
from procedure import *
#from procedure import loadonesource

/mnt/home/bbhattarai/.local/lib/python3.7/site-packages/galpy/util/bovy_coords.py:6: FutureWarning: galpy.util.bovy_coords is being deprecated in favor of galpy.util.coords; all functions in there are the same; please switch to the new import, because the old import will be removed in v1.9
  warnings.warn('galpy.util.bovy_coords is being deprecated in favor of galpy.util.coords; all functions in there are the same; please switch to the new import, because the old import will be removed in v1.9',FutureWarning)

PLEASE IMPORT superfreq: pip install --user git+https://github.com/adrn/superfreq
start at: 2021-08-10 09:13:06.625469
end at: 2021-08-10 09:13:06.625918


In [2]:
!hostname
!pwd

pcn-9-71
/mnt/home/bbhattarai


In [3]:
datapath="/mnt/home/bbhattarai/B3-N/"
freq_datapath="/mnt/home/bbhattarai/ceph/freq_data/"
plotpath="/mnt/home/bbhattarai/resonance_sweeping/New_Sims_Analysis/plots/"

In [4]:
snapshot=800

In [5]:
freq_datafile="DiskActions"+str(snapshot)+"_B3N.npy"

In [6]:
freqs= np.load(freq_datapath+freq_datafile)

# Reading the values from the frequency data

In [7]:
#Jrdisk=freqs[0]
#Jphidisk=freqs[1]
#Jzdisk=freqs[2]
#Trdisk=freqs[3]
#Tphidisk=freqs[4]
#Tzdisk=freqs[5]
Ordisk=freqs[6]
Ophidisk=freqs[7]
#Ozdisk=freqs[8]
idd_from_freqs=freqs[9]
del freqs

# Reading the snapshot data

In [8]:
snaparr = loadwholesnap(path,snapshot)
print("These are the data we have",snaparr[0].dtype)
idd=snaparr['idd']
x=snaparr['x']
y=snaparr['y']
z=snaparr['z']
vx=snaparr['vx']
vy=snaparr['vy']
#vz=snaparr['vz']  
mass=snaparr['mass']  #mass is in solar mass (change old mass calculations to take account of the factor 2.324876e9)

vr=snaparr['vr']
vphi=snaparr['vphi']
#vzz=snaparr['vzz']
r=snaparr['r']
phi=snaparr['phi'] #phi is in radians
#zz=snaparr['zz']

#converting phi to degrees
phi_degrees=np.rad2deg(phi)
del snaparr

These are the data we have [('t', '<f8'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4'), ('vr', '<f4'), ('vphi', '<f4'), ('vzz', '<f4'), ('r', '<f4'), ('phi', '<f4'), ('zz', '<f4'), ('mass', '<f4'), ('idd', '<u8')]


# Making the cut

In [9]:
discindx=(mass<1e-7*2.324876e9)
idd_snapshots=idd[discindx]
x_select=x[discindx]
y_select=y[discindx]
z_select=z[discindx]

vx_select=vx[discindx]
vy_select=vy[discindx]
#vz_select=vz[discindx]


vr_select=vr[discindx]
vphi_select=vphi[discindx]
#vzz_select=vzz[discindx]


r_select=r[discindx]
phi_select=phi_degrees[discindx]

In [10]:
del discindx
del mass
print("These are the ids obtained from the frequency data",len(idd_from_freqs))
print("These are the ids obtained from the snapshot  data",len(idd_snapshots))

These are the ids obtained from the frequency data 250980160
These are the ids obtained from the snapshot  data 250980160


# Loading all saved barangles

In [11]:
angle_datapath="/mnt/home/bbhattarai/resonance_sweeping//New_Sims_Analysis/"
datafilename="0_to_1048_B3-N_fft_barangles_combined_degrees.ang"
ang_stored = open(angle_datapath+datafilename,'rb')
all_bangles=pickle.load(ang_stored)
print(len(all_bangles))

1048


# Checking resonance Conditions

In [12]:
#datafilename="saved_bar_pattern_speed_km_per_s_kpc.ang"
datafilename="B3-N_saved_bar_pattern_speed_km_per_s_kpc.ang"
save_datapath="/mnt/home/bbhattarai/resonance_sweeping//New_Sims_Analysis/"
ps_stored = open(save_datapath+datafilename,'rb')
ps=pickle.load(ps_stored)
#print(ps)
ps_located=ps[snapshot]
print(ps_located)

31.16421617364574


# Adjusting pattern_speed_value

In [13]:
start=snapshot-3
end=snapshot+3
s=0
c=0
for i in range(start,end):
    c+=1
    s=s+ps[i]
print(c)
ps_adjusted=s/c

print(ps_adjusted)

#factor=3.08567758/3.15576
#ps_adjusted=ps_adjusted/factor
print(ps_adjusted)

6
31.14740421816325
31.14740421816325


# Calculating the Resonance Conditions

In [14]:
#OLR resonance
#olr_resonance=Ophidisk-Ordisk/2 # inner Linbald resonance

olr_resonance=Ophidisk+Ordisk/2 # outer Linbald resonance
omega_diff_olr=olr_resonance-ps_adjusted
keep_olr=(omega_diff_olr<0.1)*(omega_diff_olr>-0.1)


#CR resonance
omega_diff=Ophidisk-ps_adjusted
keep_cr=(omega_diff<0.1)*(omega_diff>-0.1)

# Entire Galactic Regions with Co-Rotating (CR) Resonance

In [15]:
x_resonance_cr=(x_select[keep_cr])
y_resonance_cr=(y_select[keep_cr])
z_resonance_cr=(z_select[keep_cr])
vr_resonance_cr=(vr_select[keep_cr])
vphi_resonance_cr=(vphi_select[keep_cr])
#vzz_resonance_cr=(vzz_select[keep_cr])

print(len(x_resonance_cr))

r_resonance_cr=r_select[keep_cr]
phi_resonance_cr=phi_select[keep_cr]

427532


# checking for resonance stars from snapshot 700

In [16]:
idd_resonance_cr=idd_snapshots[keep_cr]

datafilename="700idd_resonance_cr.pkl"

idd_stored = open(save_datapath+datafilename,'rb')
idd_resonance_cr_700=pickle.load(idd_stored)

In [17]:
commons=list()
for i in range(len(idd_resonance_cr_700)):
    c=np.where(idd_resonance_cr==idd_resonance_cr_700[i])
    commons.extend(list(idd_resonance_cr[c]))
print(len(commons))



5815


# Calculating the percentage of stars that are in resonance with the bar:

In [18]:
n_star=len(x_select)
n_star_cr=len(x_resonance_cr)

print(n_star)
print(n_star_cr)
percentage_cr=(n_star_cr/n_star)*100
print("The percentage of stars that are in Corotation Resonance is",percentage_cr)

250980160
427532
The percentage of stars that are in Corotation Resonance is 0.17034493881906843


# Entire Galactic Regions with OLR (Outer Linbald Resonance) Resonance

In [19]:
x_resonance_olr=(x_select[keep_olr])
y_resonance_olr=(y_select[keep_olr])
z_resonance_olr=(z_select[keep_olr])
vr_resonance_olr=(vr_select[keep_olr])
vphi_resonance_olr=(vphi_select[keep_olr])
#vzz_resonance_olr=(vzz_select[keep_olr])

r_resonance_olr=r_select[keep_olr]
phi_resonance_olr=phi_select[keep_olr]

n_star_olr=len(x_resonance_olr)
print(n_star_olr)
percentage_olr=(n_star_olr/n_star)*100
print("The percentage of stars that are in OLR are: ",percentage_olr)


1440154
The percentage of stars that are in OLR are:  0.5738118901509984


# looking for common stars_from snapshot 700

In [20]:
idd_resonance_olr=idd_snapshots[keep_olr]

datafilename="700idd_resonance_olr.pkl"

idd_stored_olr = open(save_datapath+datafilename,'rb')
idd_resonance_olr_700=pickle.load(idd_stored_olr)

commons_olr=list()
for i in range(len(idd_resonance_olr_700)):
    c=np.where(idd_resonance_olr==idd_resonance_olr_700[i])
    commons_olr.extend(list(idd_resonance_olr[c]))
print(len(commons_olr))


34560


# Selecting the slice -5<phi<5

In [21]:
keep_phi_slice_cr=(phi_resonance_cr<5)*(phi_resonance_cr>-5)
keep_phi_slice_olr=(phi_resonance_olr<5)*(phi_resonance_olr>-5)

phi_slice_cr=phi_resonance_cr[keep_phi_slice_cr]
print(len(phi_slice_cr))

phi_slice_olr=phi_resonance_olr[keep_phi_slice_olr]
print(len(phi_slice_olr))

11243
46482


# Keeping record of values for the slice -5<phi<5

In [22]:
keep_phi_slice=(phi_select<5)*(phi_select>-5)
x_select_phi_slice=x_select[keep_phi_slice]
y_select_phi_slice=y_select[keep_phi_slice]
z_select_phi_slice=z_select[keep_phi_slice]

vr_phi_slice=vr_select[keep_phi_slice]
vphi_phi_slice=vphi_select[keep_phi_slice]
#vzz_phi_slice=vzz_select[keep_phi_slice]
r_phi_slice=r_select[keep_phi_slice]



# Keeping record of values in the Solar Neighborhood

In [23]:
keep_sn=(x_select<8.5)*(x_select>7.5)*(y_select<0.5)*(y_select>-0.5)*(z_select<0.5)*(z_select>-0.5)
x_select_sn=x_select[keep_sn]
y_select_sn=y_select[keep_sn]
z_select_sn=z_select[keep_sn]

vr_sn=vr_select[keep_sn]
vphi_sn=vphi_select[keep_sn]
#vzz_sn=vzz_select[keep_sn]
r_sn=r_select[keep_sn]

# Corotating Resonance (CR) in the slice

In [24]:
x_resonance_cr_phi_slice=x_resonance_cr[keep_phi_slice_cr]
y_resonance_cr_phi_slice=(y_resonance_cr[keep_phi_slice_cr])
z_resonance_cr_phi_slice=(z_resonance_cr[keep_phi_slice_cr])
vr_resonance_cr_phi_slice=(vr_resonance_cr[keep_phi_slice_cr])
vphi_resonance_cr_phi_slice=(vphi_resonance_cr[keep_phi_slice_cr])
#vzz_resonance_cr_phi_slice=(vzz_resonance_cr[keep_phi_slice_cr])

r_resonance_cr_phi_slice=r_resonance_cr[keep_phi_slice_cr]

phi_resonance_cr_phi_slice=phi_resonance_cr[keep_phi_slice_cr]

print(x_resonance_cr_phi_slice)

[6.327705  5.820906  7.1321225 ... 4.6087475 4.678745  4.4224153]


# checking the matching stars with CR in the slice from snapshot 700

In [25]:
idd_resonance_cr_phi_slice_1000=idd_resonance_cr[keep_phi_slice_cr]

datafilename="700idd_resonance_cr_phi_slice.pkl"

idd_stored_phi_slice_cr = open(save_datapath+datafilename,'rb')
idd_resonance_phi_slice_cr_700=pickle.load(idd_stored_phi_slice_cr)

commons_phi_slice_cr=list()
for i in range(len(idd_resonance_phi_slice_cr_700)):
    c=np.where(idd_resonance_cr_phi_slice_1000==idd_resonance_phi_slice_cr_700[i])
    commons_phi_slice_cr.extend(list(idd_resonance_cr_phi_slice_1000[c]))
print(len(commons_phi_slice_cr))

2


# OLR in the slice

In [26]:
x_resonance_olr_phi_slice=x_resonance_olr[keep_phi_slice_olr]
y_resonance_olr_phi_slice=(y_resonance_olr[keep_phi_slice_olr])
z_resonance_olr_phi_slice=(z_resonance_olr[keep_phi_slice_olr])
vr_resonance_olr_phi_slice=(vr_resonance_olr[keep_phi_slice_olr])
vphi_resonance_olr_phi_slice=(vphi_resonance_olr[keep_phi_slice_olr])
#vzz_resonance_olr_phi_slice=(vzz_resonance_olr[keep_phi_slice_olr])

r_resonance_olr_phi_slice=r_resonance_olr[keep_phi_slice_olr]

phi_resonance_olr_phi_slice=phi_resonance_olr[keep_phi_slice_olr]

print(x_resonance_olr_phi_slice)

[ 7.738738   6.785485  14.638245  ...  5.045147   4.8314633  4.9647775]


In [27]:
#checking for matching idds

idd_resonance_olr_phi_slice=idd_resonance_olr[keep_phi_slice_olr]

datafilename="700idd_resonance_olr_phi_slice.pkl"

idd_stored_phi_slice_olr = open(save_datapath+datafilename,'rb')
idd_resonance_phi_slice_olr_700=pickle.load(idd_stored_phi_slice_olr)

commons_phi_slice_olr=list()
for i in range(len(idd_resonance_phi_slice_olr_700)):
    c=np.where(idd_resonance_olr_phi_slice==idd_resonance_phi_slice_olr_700[i])
    commons_phi_slice_olr.extend(list(idd_resonance_olr_phi_slice[c]))
print(len(commons_phi_slice_olr))

0


# Selecting the solar neighboorhood

In [28]:
keep_cr_sn=(x_resonance_cr<8.5)*(x_resonance_cr>7.5)*(y_resonance_cr<0.5)*(y_resonance_cr>-0.5)*(z_resonance_cr<0.5)*(z_resonance_cr>-0.5)
keep_olr_sn=(x_resonance_olr<8.5)*(x_resonance_olr>7.5)*(y_resonance_olr<0.5)*(y_resonance_olr>-0.5)*(z_resonance_olr<0.5)*(z_resonance_olr>-0.5)

#keep_cr_sn=(x_resonance_cr<8.5)*(x_resonance_cr>7.5)
#keep_olr_sn=(x_resonance_olr<8.5)*(x_resonance_olr>7.5)

# CR in the Solar Neighborhood

In [29]:
x_resonance_cr_sr=x_resonance_cr[keep_cr_sn]
y_resonance_cr_sr=(y_resonance_cr[keep_cr_sn])
z_resonance_cr_sr=(z_resonance_cr[keep_cr_sn])
vr_resonance_cr_sr=(vr_resonance_cr[keep_cr_sn])
vphi_resonance_cr_sr=(vphi_resonance_cr[keep_cr_sn])
#vzz_resonance_cr_sr=(vzz_resonance_cr[keep_cr_sn])

r_resonance_cr_sr=r_resonance_cr[keep_cr_sn]

phi_resonance_cr_sr=phi_resonance_cr[keep_cr_sn]

In [30]:
#checking for matching idds

idd_resonance_cr_sr=idd_resonance_cr[keep_cr_sn]

datafilename="700idd_resonance_cr_sr.pkl"

idd_stored_cr_sr = open(save_datapath+datafilename,'rb')
idd_resonance_cr_sr_700=pickle.load(idd_stored_cr_sr)

commons_cr_sr=list()
for i in range(len(idd_resonance_cr_sr_700)):
    c=np.where(idd_resonance_cr_sr==idd_resonance_cr_sr_700[i])
    commons_cr_sr.extend(list(idd_resonance_cr_sr[c]))
print(len(commons_cr_sr))

0


# OLR in the Solar Neighborhood

In [31]:
x_resonance_olr_sr=(x_resonance_olr[keep_olr_sn])
y_resonance_olr_sr=(y_resonance_olr[keep_olr_sn])
z_resonance_olr_sr=(z_resonance_olr[keep_olr_sn])
vr_resonance_olr_sr=(vr_resonance_olr[keep_olr_sn])
vphi_resonance_olr_sr=(vphi_resonance_olr[keep_olr_sn])
#vzz_resonance_olr_sr=(vzz_resonance_olr[keep_olr_sn])

r_resonance_olr_sr=r_resonance_olr[keep_olr_sn]
phi_resonance_olr_sr=phi_resonance_olr[keep_olr_sn]

In [32]:
#checking for matching idds

idd_resonance_olr_sr=idd_resonance_olr[keep_olr_sn]

datafilename="700idd_resonance_olr_sr.pkl"

idd_stored_olr_sr = open(save_datapath+datafilename,'rb')
idd_resonance_olr_sr_700=pickle.load(idd_stored_olr_sr)

commons_olr_sr=list()
for i in range(len(idd_resonance_olr_sr_700)):
    c=np.where(idd_resonance_olr_sr==idd_resonance_olr_sr_700[i])
    commons_olr_sr.extend(list(idd_resonance_olr_sr[c]))
print(len(commons_olr_sr))

0


# Printing resonance data

In [33]:
print("Ones with OLR resonance in the solar neighborhood: ",y_resonance_olr_sr)
print("Ones with CR resonance in the solar neighborhood: ",y_resonance_cr_sr)

Ones with OLR resonance in the solar neighborhood:  [0.13539053 0.11617247 0.24033543 ... 0.49919045 0.3898266  0.39090514]
Ones with CR resonance in the solar neighborhood:  [ 2.39046827e-01  3.68401229e-01  2.71453857e-01  3.22506785e-01
  2.35890225e-01  2.35940591e-01  3.65738988e-01  3.08652550e-01
  2.43124872e-01  2.89240092e-01  2.89403558e-01  1.46443695e-01
  1.66457579e-01  9.23514962e-02  5.26732616e-02  1.71075016e-01
  1.54910281e-01  1.85734808e-01  1.04269631e-01  1.68730959e-01
  2.03729466e-01  8.73199850e-02  4.26607840e-02  5.92240281e-02
  3.47240008e-02  5.35873696e-02  7.77621120e-02  1.30406708e-01
 -4.00433801e-02 -8.19723383e-02 -9.42367166e-02 -8.36657435e-02
 -6.07733317e-02 -3.84296440e-02 -6.92857355e-02 -4.61861454e-02
 -1.46630868e-01 -2.24715129e-01 -3.11126769e-01 -2.81361550e-01
 -2.41057575e-01 -2.34927520e-01 -2.63748646e-01 -2.47508585e-01
 -2.95852721e-01 -1.98557124e-01 -2.70590901e-01 -3.06935966e-01
  2.74474639e-03 -2.82640085e-02 -2.57555068e

In [34]:
# for CR

x_700=list()
y_700=list()
for i in range(len(commons)):
    test = loadonesource(path,commons[i],start=700,finish=701)
    x_test=test['x']
    y_test=test['y']
    x_700.extend(x_test)
    y_700.extend(y_test)

print(len(x_700))


x_1000=list()
y_1000=list()
for i in range(len(commons)):
    test = loadonesource(path,commons[i],start=snapshot,finish=snapshot+1)
    x_test=test['x']
    y_test=test['y']
    x_1000.extend(x_test)
    y_1000.extend(y_test)


5815


In [35]:
# for OLR

x_700_olr=list()
y_700_olr=list()
for i in range(len(commons_olr)):
    test = loadonesource(path,commons_olr[i],start=700,finish=701)
    x_test=test['x']
    y_test=test['y']
    x_700_olr.extend(x_test)
    y_700_olr.extend(y_test)

print(len(x_700))



x_1000_olr=list()
y_1000_olr=list()
for i in range(len(commons_olr)):
    test = loadonesource(path,commons_olr[i],start=snapshot,finish=snapshot+1)
    x_test=test['x']
    y_test=test['y']
    x_1000_olr.extend(x_test)
    y_1000_olr.extend(y_test)


5815


In [36]:
print(len(x_1000_olr))
print(len(x_700_olr))

34560
34560


# Plotting resonance stars before and after

In [37]:
fig3=plt.figure()
ax=fig3.add_subplot(111)
#ax.text(8,300,r"Snapshot %s, Slice: -5<$\phi$<5"%(str(snapshot)))
#cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,C=Ordisk,vmin=1,vmax=40)
cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,bins='log')
ax.scatter(x_700_olr,y_700_olr,c="red",marker=".",s=0.01,label="OLR at 700")
ax.scatter(x_1000_olr,y_1000_olr,c="yellow",marker=".",s=0.01,label="OLR at "+str(snapshot))
ax.scatter(x_700,y_700,c="orange",marker=".",s=0.1,label="CR at 700")
ax.scatter(x_1000,y_1000,c="black",marker=".",s=0.1,label="CR at "+str(snapshot))
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend(bbox_to_anchor=(1.13,0.93), loc='center left',markerscale=9)
ax.text(11,30,r"Snapshot "+str(snapshot)+" (B3-N)")
#ax.set_xlim(7.5,8.5)
#ax.set_ylim(-0.5,0.5)
cbar_ax = fig3.add_axes([0.91, 0.12, 0.03, 0.76]) # position of the colorbar (left, bottom, width, height)
fig3.colorbar(cb1, cax=cbar_ax)
cbar_ax.set_ylabel(r'n$_{star}$')
cbar_ax.yaxis.label.set_size(10)
plotname=str(snapshot)+"B3-N_y_vs_x_with_resonance_regions_whole_galaxy_before_after.jpg"
fig3.savefig(plotpath+plotname,bbox_inches="tight",dpi=250)
print("Plot generated and saved to file: ",plotname)
plt.close()

Plot generated and saved to file:  800B3-N_y_vs_x_with_resonance_regions_whole_galaxy_before_after.jpg


# Plotting x and y of resonance regions in the entire galaxy

In [ ]:
fig4=plt.figure()
ax=fig4.add_subplot(111)
#ax.text(8,300,r"Snapshot %s, Slice: -5<$\phi$<5"%(str(snapshot)))
#cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,C=Ordisk,vmin=1,vmax=40)
cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,bins='log')
ax.scatter(x_resonance_cr,y_resonance_cr,c="black",marker=".",s=0.01,label="CR")
ax.scatter(x_resonance_olr,y_resonance_olr,c="red",marker=".",s=0.01,label="OLR")
ax.scatter(x_resonance_cr_sr,y_resonance_cr_sr,c="yellow",marker=".",s=0.1,label="CR-SN")
ax.scatter(x_resonance_olr_sr,y_resonance_olr_sr,c="orange",marker=".",s=0.1,label="OLR-SN")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend(bbox_to_anchor=(1.13,0.93), loc='center left',markerscale=9)
ax.text(11,30,r"Snapshot "+str(snapshot)+" (B3-N)")
#ax.set_xlim(7.5,8.5)
#ax.set_ylim(-0.5,0.5)
cbar_ax = fig4.add_axes([0.91, 0.12, 0.03, 0.76]) # position of the colorbar (left, bottom, width, height)
fig4.colorbar(cb1, cax=cbar_ax)
cbar_ax.set_ylabel(r'n$_{star}$')
cbar_ax.yaxis.label.set_size(10)
plotname=str(snapshot)+"B3-N_y_vs_x_with_resonance_regions_whole_galaxy.jpg"
fig4.savefig(plotpath+plotname,bbox_inches="tight",dpi=250)
print("Plot generated and saved to file: ",plotname)
plt.close()

In [ ]:
print(np.max(vr_select))

In [ ]:
fig5=plt.figure()
ax=fig5.add_subplot(111)
#ax.text(8,300,r"Snapshot %s, Slice: -5<$\phi$<5"%(str(snapshot)))
#cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,C=Ordisk,vmin=1,vmax=40)
cb1=ax.hexbin(vr,vphi,extent=((-650,650,-650,650)),gridsize=500,mincnt=1,bins='log')
ax.set_xlabel("vR")
ax.set_ylabel(r"v$\phi$")
ax.scatter(vr_resonance_cr,vphi_resonance_cr,c="black",marker=".",s=0.01,label="CR")
ax.scatter(vr_resonance_olr,vphi_resonance_olr,c="red",marker=".",s=0.01,label="OLR")
ax.scatter(vr_resonance_cr_sr,vphi_resonance_cr_sr,c="yellow",marker=".",s=0.1,label="CR-SN")
ax.scatter(vr_resonance_olr_sr,vphi_resonance_olr_sr,c="orange",marker=".",s=0.1,label="OLR-SN")
ax.legend(bbox_to_anchor=(1.13,0.93), loc='center left',markerscale=9)
ax.text(248,550,r"Snapshot "+str(snapshot)+" (B3-N)")

cbar_ax = fig5.add_axes([0.91, 0.12, 0.03, 0.76]) # position of the colorbar (left, bottom, width, height)
fig5.colorbar(cb1, cax=cbar_ax)
cbar_ax.set_ylabel(r'n$_{star}$')
cbar_ax.yaxis.label.set_size(10)
plotname=str(snapshot)+"B3-N_v_phi_vs_vr_with_resonance_regions.jpg"
fig5.savefig(plotpath+plotname,bbox_inches="tight",dpi=200)
print("Plot generated and saved to file: ",plotname)
plt.close()

In [ ]:
fig5_1=plt.figure()
ax=fig5_1.add_subplot(111)
#ax.text(8,300,r"Snapshot %s, Slice: -5<$\phi$<5"%(str(snapshot)))
#cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,C=Ordisk,vmin=1,vmax=40)
cb1=ax.hexbin(vr_sn,vphi_sn,extent=((-200,200,50,300)),gridsize=500,mincnt=1,vmin=1,vmax=30)
ax.set_xlabel("vR")
ax.set_ylabel(r"v$\phi$")
#ax.scatter(vr_resonance_cr,vphi_resonance_cr,c="black",marker=".",s=0.01,label="CR")
#ax.scatter(vr_resonance_olr,vphi_resonance_olr,c="red",marker=".",s=0.01,label="OLR")
ax.scatter(vr_resonance_cr_sr,vphi_resonance_cr_sr,c="yellow",marker="*",s=0.3,label="CR-SN")
ax.scatter(vr_resonance_olr_sr,vphi_resonance_olr_sr,c="orange",marker="*",s=0.3,label="OLR-SN")
ax.legend(bbox_to_anchor=(1.13,0.93), loc='center left',markerscale=9)
ax.text(70,285,r"Snapshot "+str(snapshot)+" (B3-N)")
ax.text(-180,285,r"7.5<x<8.5, -0.5<y<0.5, -0.5<z<0.5")

cbar_ax = fig5_1.add_axes([0.91, 0.12, 0.03, 0.76]) # position of the colorbar (left, bottom, width, height)
fig5_1.colorbar(cb1, cax=cbar_ax)
cbar_ax.set_ylabel(r'n$_{star}$')
cbar_ax.yaxis.label.set_size(10)
plotname=str(snapshot)+"B3-N_v_phi_vs_vr_with_resonance_region_in_sn_slice.jpg"
fig5_1.savefig(plotpath+plotname,bbox_inches="tight",dpi=300)
print("Plot generated and saved to file: ",plotname)
plt.close()

# Selecting a small angular region for analysis

In [ ]:
keep_phi=(phi_select<5)*(phi_select>-5)
vphi_slice=vphi_select[keep_phi]

r_slice=r_select[keep_phi]

fig6=plt.figure()
ax=fig6.add_subplot(212)
ax1=fig6.add_subplot(211)
#ax.text(8,300,r"Snapshot %s, Slice: -5<$\phi$<5"%(str(snapshot)))
#cb1=ax.hexbin(x_select,y_select,extent=((-35,35,-35,35)),gridsize=500,mincnt=1,C=Ordisk,vmin=1,vmax=40)
cb1=ax.hexbin(r_slice,vphi_slice,extent=((5,15,50,300)),gridsize=500,mincnt=1,vmin=1,vmax=80)
ax.set_xlabel("R")
ax.set_ylabel(r"v$\phi$")
ax_cr=ax.scatter(r_resonance_cr_phi_slice,vphi_resonance_cr_phi_slice,c="black",marker=".",s=0.1,label="CR")
print(len(r_resonance_cr))
ax_olr=ax.scatter(r_resonance_olr_phi_slice,vphi_resonance_olr_phi_slice,c="red",marker=".",s=0.1,label="OLR")
ax.scatter(r_resonance_cr_sr,vphi_resonance_cr_sr,c="yellow",marker="*",s=0.1,label="CR-SN")
ax.scatter(r_resonance_olr_sr,vphi_resonance_olr_sr,c="orange",marker="*",s=0.1,label="OLR-SN")

cbar_ax = fig6.add_axes([0.91, 0.12, 0.03, 0.76]) # position of the colorbar (left, bottom, width, height)
fig6.colorbar(cb1, cax=cbar_ax)
cbar_ax.set_ylabel(r'n$_{star}$')
cbar_ax.yaxis.label.set_size(10)
ax.set_xlim(5,15)
ax.set_ylim(50,300)
#ax.legend(bbox_to_anchor=(0.77,0.93), loc='center left',markerscale=9)
ax.legend(bbox_to_anchor=(1.13,1.6), loc='center left',markerscale=9)

ax1.hexbin(r_slice,vphi_slice,extent=((5,15,50,300)),gridsize=500,mincnt=1,vmin=1,vmax=80)
ax1.set_xlim(5,15)
ax1.set_ylim(50,300)
ax1.text(12,270,r"Slice -5<$\phi$<5")
#ax.legend(markerscale=6)
plotname=str(snapshot)+"B3-N_v_phi_vs_r_with_resonance_regions.jpg"
fig6.suptitle("B3-N, Snapshot "+str(snapshot))
fig6.savefig(plotpath+plotname,bbox_inches="tight",dpi=300)
print("Plot generated and saved to file: ",plotname)
plt.close()

In [ ]:
print(r_resonance_cr_sr)

# Loading new sims demo !!!!

In [ ]:
from procedure import *

In [ ]:
path

In [ ]:
snaparr = loadwholesnap(path,90)

In [ ]:
print(snaparr[0].dtype)

In [ ]:
snaparr[(snaparr['x']>3)&(snaparr['y']<2)]['z']

In [ ]:
np.where(snaparr['idd']==241071075)[0]